pm10 데이터 전처리

In [1]:
import pandas as pd

# 1. CSV 파일 읽기
input_path = 'pm10.csv'
df = pd.read_csv(input_path, encoding='cp949')

# 2. 지역명 합치기 (조건부로 처리)
special_cities = ['서울특별시', '부산광역시', '대구광역시', '인천광역시',
                  '광주광역시', '대전광역시', '울산광역시', '세종특별자치시']
df['지역'] = df.apply(
    lambda row: row['구분(1)'] if row['구분(1)'] in special_cities
    else row['구분(1)'] + ' ' + row['구분(2)'],
    axis=1
)

# 총계/소계 행 제거
df = df[~df['지역'].str.contains('총계|소계', na=False)]
df = df.drop(columns=['구분(1)', '구분(2)'])

# 3. * 제거 및 숫자 변환
for col in df.columns:
    if col != '지역':
        df[col] = df[col].astype(str).str.replace('*', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 4. 데이터 구조 변경: wide → long
df_long = df.melt(id_vars='지역', var_name='월', value_name='PM10')

# 5. 월 컬럼에서 'MM' 추출
df_long['월숫자'] = df_long['월'].str.extract(r'\.(\d+)$')

# 6-1. 1차 보간: 해당 지역의 같은 월 평균
df_long['PM10'] = df_long.groupby(['지역', '월숫자'])['PM10'].transform(lambda x: x.fillna(x.mean()))

# 6-2. 2차 보간: 전체 지역의 해당 월 평균 (여전히 남은 결측치)
df_long['PM10'] = df_long.groupby('월')['PM10'].transform(lambda x: x.fillna(x.mean()))

# 7. 소수점 제거
df_long['PM10'] = df_long['PM10'].round(0).astype('Int64')

# 8. 다시 wide 포맷으로 되돌리기
df_cleaned = df_long.pivot(index='지역', columns='월', values='PM10').reset_index()

# 9. 열 순서 정렬
month_order = sorted([col for col in df_cleaned.columns if col != '지역'])
df_cleaned = df_cleaned[['지역'] + month_order]

# 10. 저장
output_path = 'pm10_cleaned.csv'
df_cleaned.to_csv(output_path, index=False, encoding='utf-8-sig')



pm2.5 데이터 전처리

In [2]:
import pandas as pd

# 1. CSV 파일 읽기
file_path = 'pm25.csv'
df = pd.read_csv(file_path, encoding='cp949')

# 2. 지역명 합치기 (조건부로 처리)
special_cities = ['서울특별시', '부산광역시', '대구광역시', '인천광역시',
                  '광주광역시', '대전광역시', '울산광역시', '세종특별자치시']
df['지역'] = df.apply(
    lambda row: row['구분(1)'] if row['구분(1)'] in special_cities
    else row['구분(1)'] + ' ' + row['구분(2)'],
    axis=1
)

# 총계/소계 행 제거
df = df[~df['지역'].str.contains('총계|소계', na=False)]
df = df.drop(columns=['구분(1)', '구분(2)'])

# 3. * 제거 및 숫자 변환
for col in df.columns:
    if col != '지역':
        df[col] = df[col].astype(str).str.replace('*', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 4. 데이터 구조 변경: wide → long
df_long = df.melt(id_vars='지역', var_name='월', value_name='PM25')

# 5. 월 컬럼에서 'MM' 추출
df_long['월숫자'] = df_long['월'].str.extract(r'\.(\d+)$')

# 6-1. 1차 보간: 해당 지역의 같은 월 평균
df_long['PM25'] = df_long.groupby(['지역', '월숫자'])['PM25'].transform(lambda x: x.fillna(x.mean()))

# 6-2. 2차 보간: 전체 지역의 해당 월 평균 (여전히 남은 결측치)
df_long['PM25'] = df_long.groupby('월')['PM25'].transform(lambda x: x.fillna(x.mean()))

# 7. 소수점 제거
df_long['PM25'] = df_long['PM25'].round(0).astype('Int64')

# 8. 다시 wide 포맷으로 되돌리기
df_cleaned = df_long.pivot(index='지역', columns='월', values='PM25').reset_index()

# 9. 열 순서 정렬
month_order = sorted([col for col in df_cleaned.columns if col != '지역'])
df_cleaned = df_cleaned[['지역'] + month_order]

# 10. 저장
output_path = 'pm25_cleaned.csv'
df_cleaned.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 완료: {output_path} 로 저장되었습니다.")


✅ 완료: pm25_cleaned.csv 로 저장되었습니다.


pm2.5 + pm10 데이터 병합 및, 총 점수 계산(pm2.5*3 + pm10)

In [3]:
import pandas as pd

# 파일 읽기
pm25 = pd.read_csv('pm25_cleaned.csv', encoding='utf-8')
pm10 = pd.read_csv('pm10_cleaned.csv', encoding='utf-8')

# 컬럼명 구분을 위해 접두사 추가
pm25 = pm25.rename(columns={col: f'PM25_{col}' for col in pm25.columns[1:]})
pm10 = pm10.rename(columns={col: f'PM10_{col}' for col in pm10.columns[1:]})

# 지역 기준으로 병합
merged = pd.merge(pm25, pm10, on='지역', how='left')

# 전처리
merged = merged.applymap(lambda x: x if not isinstance(x, (int, float)) or x >= 0 else None)
merged = merged.ffill(axis=0)
merged = merged.apply(pd.to_numeric, errors='ignore')
merged['지역'] = merged['지역'].astype(str)

# 월별 컬럼 계산
months = [col.split('_')[1] for col in merged.columns if 'PM25_' in col]
for month in months:
    pm25_col = f'PM25_{month}'
    pm10_col = f'PM10_{month}'
    merged[month] = merged[pm25_col] * 3 + merged[pm10_col]

# 기존 PM25/PM10 컬럼 제거
merged.drop(columns=[f'PM25_{m}' for m in months], inplace=True)
merged.drop(columns=[f'PM10_{m}' for m in months], inplace=True)

merged['지역'] = merged['지역'].str.replace("전북특별자치도", "전라북도")
merged['지역'] = merged['지역'].str.replace("강원특별자치도", "강원도")
merged['지역'] = merged['지역'].str.replace("울릉도", "울릉")
# 컬럼 순서 정렬
merged = merged[['지역'] + months]

# 저장
merged.to_csv('WhoMergedPm.csv', index=False, encoding='utf-8-sig')


C:\Users\080a3\AppData\Local\Temp\ipykernel_27832\1092835110.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged = merged.applymap(lambda x: x if not isinstance(x, (int, float)) or x >= 0 else None)
C:\Users\080a3\AppData\Local\Temp\ipykernel_27832\1092835110.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  merged = merged.apply(pd.to_numeric, errors='ignore')


인구 데이터 전처리

In [4]:
import pandas as pd
# 1. 파일 불러오기
input_path = "population.csv"
df_population = pd.read_csv(input_path, encoding='euc-kr')

# 2. 열 이름 정리
df_population.columns = df_population.columns.str.strip()

# 3. 필요한 열만 추출
df_population = df_population[['행정구역', '2020년04월_총인구수']].copy()

# 4. 괄호 제거 (행정코드)
df_population['행정구역'] = df_population['행정구역'].str.replace(r'\s*\(.*\)', '', regex=True)

# 5. 숫자형 변환
df_population['2020년04월_총인구수'] = df_population['2020년04월_총인구수'].str.replace(',', '').astype(int)

# 6. 고유행정구역명 생성
df_population['고유행정구역명'] = df_population['행정구역'].str.strip()

population_ready = df_population[['고유행정구역명', '2020년04월_총인구수']]

# 저장
population_ready.to_csv("population_cleaned.csv", index=False, encoding='utf-8-sig')

병원 데이터 전처리-1 (구가 있는 시들의 데이터를 하나로 합침)

In [5]:
# 1. 파일 불러오기
input_path = "./hospital.xlsx"
df_raw = pd.read_excel(input_path, sheet_name=0, header=None)

# 2. 헤더 설정
df_new = df_raw[3:].reset_index(drop=True)
df_new.columns = df_raw.iloc[2]

# 3. 통합할 도시 정의
city_patterns = {
    "경기도 수원시": r"^수원.*구$",
    "경기도 성남시": r"^성남.*구$",
    "경기도 안산시": r"^안산.*구$",
    "경기도 고양시": r"^고양.*구$",
    "경기도 용인시": r"^용인.*구$",
    "경기도 안양시": r"^안양.*구$",
    "충청북도 청주시": r"^청주.*구$",
    "충청남도 천안시": r"^천안.*구$",
    "경상북도 포항시": r"^포항.*구$",
    "경상남도 창원시": r"^창원.*구$",
    "전라북도 전주시": r"^전주.*구$"
}

# 4. 구가 존재하는 시들의 행을 병합함
df_city_merge = df_new.copy()
merged_city_rows = []

for city_fullname, pattern in city_patterns.items():
    gu_rows = df_city_merge[df_city_merge['시군구'].str.contains(pattern, na=False)].copy()
    if gu_rows.empty:
        continue

    numeric_cols = gu_rows.select_dtypes(include='number').columns
    gu_sum = gu_rows[numeric_cols].sum()

    rep_row = gu_rows.iloc[0].copy()
    rep_row['시도'] = city_fullname
    rep_row['시군구'] = city_fullname.split()[-1]  # 시 이름만
    for col in numeric_cols:
        rep_row[col] = gu_sum[col]

    merged_city_rows.append(rep_row)
    df_city_merge = df_city_merge[~df_city_merge['시군구'].str.contains(pattern, na=False)]

# 5. 병합 및 저장
df_city_merged_final = pd.concat([df_city_merge, pd.DataFrame(merged_city_rows)], ignore_index=True)
df_city_merged_final.to_csv("./hospital_combined.csv", index=False, encoding="utf-8-sig")



병원 데이터 전처리 - 2 필요한 컬럼만 추출하여 합치고, 행정구역명 컬럼 통일

In [6]:
# 1. 병원 데이터 파일 불러오기
input_path = "hospital_combined.csv" 
hospital_excel = pd.read_csv(input_path)

# 1. 필요한 열 필터링 및 헤더 정리
df_hospital_cleaned = df_city_merged_final.copy()

# 2. 필요한 열만 추출
columns_needed = ['시도', '시군구', '상급종합병원', '종합병원', '병원', '의원', '보건소', '보건지소', '보건진료소']
df_hospital_cleaned = df_hospital_cleaned[columns_needed]

# 3. 숫자형 변환
for col in columns_needed[2:]:
    df_hospital_cleaned[col] = pd.to_numeric(df_hospital_cleaned[col].replace('-', pd.NA), errors='coerce')

# 4. 고유행정구역명 생성
df_hospital_cleaned['고유행정구역명'] = df_hospital_cleaned['시도'].str.strip()

# 5. 총 의료기관 수 계산(계산에 필요한 컬럼들만 사용)
df_hospital_cleaned['총의료기관수'] = df_hospital_cleaned[
    ['상급종합병원', '종합병원', '병원', '의원', '보건소', '보건지소', '보건진료소']
].sum(axis=1, skipna=True)

# 6. 저장
hospital_ready = df_hospital_cleaned[['고유행정구역명', '총의료기관수']]
output_path = "hospital_cleaned.csv"
hospital_ready.to_csv(output_path, index=False, encoding='utf-8-sig')


면적 데이터 전처리

In [7]:
import pandas as pd

# 1. 파일 경로 설정
input_path = "area.csv"
df = pd.read_csv(input_path)

# 3. 지역 코드 제거 및 고유행정구역명 생성
df['고유행정구역명'] = df['소재지(시군구)별(1)'].str.replace(r'\s*\(.*?\)', '', regex=True).str.strip()

# 4. 면적(㎡) → 면적(㎢)로 변환 및 컬럼명 변경
df['면적_km2'] = df['면적 (㎡)'] / 1_000_000

# 5. 필요한 컬럼만 추출
df_area_cleaned = df[['고유행정구역명', '면적_km2']]

# 6. 최종 정리 및 저장
area_ready = df_area_cleaned[['고유행정구역명', '면적_km2']]
output_path = "area_cleaned.csv"
area_ready.to_csv(output_path, index=False, encoding='utf-8-sig')

고령자 비율 전처리

In [9]:
import pandas as pd

# 1. 파일 불러오기
input_path = "elderly_pop.csv"

df = pd.read_csv(input_path, encoding='euc-kr')

# 2. 헤더 및 데이터 정리
df_elderly_cleaned = df.iloc[1:].copy()
df_elderly_cleaned.columns = ['시도', '시군구', '고령인구비율(%)', '고령인구수', '전체인구수']

# 3. 고유행정구역명 생성 및 전처리
df_elderly_cleaned['시도'] = df_elderly_cleaned['시도'].str.strip()
df_elderly_cleaned['시군구'] = df_elderly_cleaned['시군구'].str.strip()
df_elderly_cleaned['고유행정구역명'] = df_elderly_cleaned['시도'] + " " + df_elderly_cleaned['시군구']

# 4. '소계' 처리
# '시군구'가 '소계'이면 고유명에서 ' 소계' 제거
df_elderly_cleaned.loc[df_elderly_cleaned['시군구'] == '소계', '고유행정구역명'] = \
    df_elderly_cleaned.loc[df_elderly_cleaned['시군구'] == '소계', '고유행정구역명'].str.replace(' 소계', '', regex=False)

# 5. 특별자치도 → 도로 통일
df_elderly_cleaned['고유행정구역명'] = df_elderly_cleaned['고유행정구역명'].str.replace("전북특별자치도", "전라북도")
df_elderly_cleaned['고유행정구역명'] = df_elderly_cleaned['고유행정구역명'].str.replace("강원특별자치도", "강원도")

# 6. 고령인구비율 숫자형 변환
df_elderly_cleaned['고령인구비율'] = pd.to_numeric(df_elderly_cleaned['고령인구비율(%)'], errors='coerce')


# . 적용 및 불필요 행 제거
df_elderly_final = df_elderly_cleaned[['고유행정구역명', '고령인구비율']].dropna()

# 9. 저장
output_path = "elderly_pop_cleaned.csv"
df_elderly_final.to_csv(output_path, index=False, encoding="utf-8-sig")



전처리된 데이터들을 통해 기존 의료취약지역과 비교하여, 회귀분석을 이용해 의료취약지역 점수를 도출

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 1. 파일 불러오기
df_hospital = pd.read_csv("hospital_cleaned.csv", encoding='utf-8-sig')
df_population = pd.read_csv("population_cleaned.csv", encoding='utf-8-sig')
df_area = pd.read_csv("area_cleaned.csv", encoding='utf-8-sig')
df_elderly = pd.read_csv("elderly_pop_cleaned.csv", encoding='utf-8-sig')

# 2. 컬럼명 통일
df_hospital.columns = ['지역', '총의료기관수']
df_population.columns = ['지역', '인구수']
df_area.columns = ['지역', '면적_km2']
df_elderly.columns = ['지역', '고령인구비율']

# 3. 병합
merged = pd.merge(df_hospital, df_population, on='지역', how='inner')
merged = pd.merge(merged, df_area, on='지역', how='inner')
merged = pd.merge(merged, df_elderly, on='지역', how='left')

# 4. 지표 계산
merged['인구10k명당_의료기관수'] = (merged['총의료기관수'] / merged['인구수']) * 10000
merged['면적1km2당_의료기관수'] = merged['총의료기관수'] / merged['면적_km2']

# 5. 정규화 (Min-Max)
merged['인구기준점수'] = (
    (merged['인구10k명당_의료기관수'] - merged['인구10k명당_의료기관수'].min()) /
    (merged['인구10k명당_의료기관수'].max() - merged['인구10k명당_의료기관수'].min())
)

merged['면적기준점수'] = (
    (merged['면적1km2당_의료기관수'] - merged['면적1km2당_의료기관수'].min()) /
    (merged['면적1km2당_의료기관수'].max() - merged['면적1km2당_의료기관수'].min())
)

merged['고령기준점수'] = 1 - (
    (merged['고령인구비율'] - merged['고령인구비율'].min()) /
    (merged['고령인구비율'].max() - merged['고령인구비율'].min())
)

# 6. 실제 취약지역 레이블
vulnerable_regions = ["군위군", "강화군", "옹진군", "가평군", "동두천시", "양평군", "여주시", "연천군",
    "고성군", "동해시", "삼척시", "속초시", "양구군", "양양군", "영월군", "인제군", "정선군",
    "철원군", "태백시", "평창군", "홍천군", "화천군", "횡성군", "괴산군", "단양군", "보은군",
    "영동군", "옥천군", "음성군", "진천군", "충주시", "공주시", "금산군", "당진시", "보령시",
    "부여군", "서산시", "서천군", "예산군", "청양군", "태안군", "홍성군", "고창군", "남원시",
    "무주군", "부안군", "순창군", "임실군", "장수군", "정읍시", "진안군", "강진군", "고흥군",
    "곡성군", "구례군", "나주시", "담양군", "무안군", "보성군", "신안군", "영광군", "영암군",
    "완도군", "장성군", "장흥군", "진도군", "함평군", "해남군", "고령군", "문경시", "봉화군",
    "상주시", "성주군", "영덕군", "영주시", "영양군", "영천시", "예천군", "울릉군", "울진군",
    "의성군", "청도군", "청송군", "거제시", "거창군", "고성군", "남해군", "밀양시", "사천시",
    "산청군", "의령군", "창녕군", "통영시", "하동군", "함안군", "함양군", "합천군", "서귀포시"]  # 실제 취약지역에서 강원도, 전라도 고성군이 중복되어 하나는 제거
merged['실제취약지역'] = merged['지역'].apply(lambda x: int(any(v in x for v in vulnerable_regions)))

# 7. 로지스틱 회귀로 가중치 학습
X = merged[['인구기준점수', '면적기준점수', '고령기준점수']]
X_clean = X.dropna()
y_clean = merged.loc[X_clean.index, '실제취약지역']

X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, stratify=y_clean, test_size=0.2, random_state=123)

model = LogisticRegression()
model.fit(X_train, y_train)

w1, w2, w3 = model.coef_[0]
print(f"학습된 가중치 → 인구: {w1:.3f}, 면적: {w2:.3f}, 고령: {w3:.3f}") # 기존 의료취약지역에 맞춰 가중치를 설정하니, 넓은 지역에 인구가 적게 살아 의료기관 수/인구수의 상관관계는 오히려 역으로 나타남

# 8. 정규화 후 점수 계산
total = abs(w1) + abs(w2) + abs(w3) #기존 의료취약지역에 맞춰 유의미하게 가중치를 사용하기 위해 절댓값으로 계산
merged['의료접근_최종점수'] = (
    abs(w1) / total * merged['인구기준점수'] +
    abs(w2) / total * merged['면적기준점수'] +
    abs(w3) / total * merged['고령기준점수']
).round(3)

# 9. 의료취약지역 분류 (하위 60.8%)
threshold = merged['의료접근_최종점수'].quantile(0.608)
merged['예측취약지역'] = merged['의료접근_최종점수'] <= threshold

# 10. 평가 지표
y_true = y_clean
y_pred = merged.loc[X_clean.index, '예측취약지역']

print("\n정확도:", accuracy_score(y_true, y_pred))
print("\n분류 보고서:\n", classification_report(y_true, y_pred))
print("\n 혼동 행렬:\n", confusion_matrix(y_true, y_pred))

# 11. 저장
merged.to_csv("tmp_score_result.csv", index=False, encoding='utf-8-sig')


학습된 가중치 → 인구: 1.087, 면적: -1.059, 고령: -4.087

정확도: 0.8819875776397516

분류 보고서:
               precision    recall  f1-score   support

           0       0.87      0.83      0.85        66
           1       0.89      0.92      0.90        95

    accuracy                           0.88       161
   macro avg       0.88      0.87      0.88       161
weighted avg       0.88      0.88      0.88       161


 혼동 행렬:
 [[55 11]
 [ 8 87]]
